# IBM Applied Data Science Capstone
## Part 1: Data Collection

**Objective:** Collect and load automotive sales data for analysis

---


## 1. Import Required Libraries


In [1]:
import pandas as pd
import numpy as np
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path
sys.path.append('../src')

from generate_data import generate_automotive_sales_data

print("Libraries imported successfully!")


Libraries imported successfully!
